In [ ]:
# mlrun: start-code

from mlrun.feature_store.api import ingest
def ingest_handler(context):
    ingest(mlrun_context=context) # The handler function must call ingest with the mlrun_context
    
def my_spark_func(df, context=None):
    return df.filter("bid>55") # PySpark code

# mlrun: end-code

In [ ]:
# This section has to be invoked once per MLRun/Iguazio upgrade
# from mlrun.runtimes import RemoteSparkRuntime
# RemoteSparkRuntime.deploy_default_image()

In [ ]:
from mlrun.datastore.sources import CSVSource
from mlrun import code_to_function
import mlrun.feature_store as fstore

feature_set = fstore.FeatureSet("stock-quotes", entities=[fstore.Entity("name")], engine="spark")

source = CSVSource("mycsv", path="v3io:///projects/quotes.csv")

feature_set.graph.to(name="s1", handler="my_spark_func")

my_func = code_to_function("func", kind="spark")

# Must-have coonfigurations:
my_func.with_driver_requests(cpu="200m", mem="1g")
my_func.with_executor_requests(cpu="200m", mem="1g")
my_func.with_igz_spark()

# Enables using the default image (can be replace with specifying a specific image with .spec.image)
my_func.spec.use_default_image = True

# Not a must - default: 1
my_func.spec.replicas = 2

config = fstore.RunConfig(local=False, function=my_func, handler="ingest_handler")

fstore.ingest(feature_set, source, run_config=config)